# Topic Modelling using LDA.

In [1]:
import pandas as pd

In [2]:
npr = pd.read_csv("npr.csv")
npr.head()

,Article
0,"In the Washington of 2016, even when the polic..."
1,Donald Trump has used Twitter — his prefe...
2,Donald Trump is unabashedly praising Russian...
3,"Updated at 2:50 p. m. ET, Russian President Vl..."
4,"From photography, illustration and video, to d..."


In [3]:
npr.info()

<class 'pandas.core.frame.DataFrame'>
RangeIndex: 200 entries, 0 to 199
Data columns (total 1 columns):
 #   Column   Non-Null Count  Dtype 
---  ------   --------------  ----- 
 0   Article  200 non-null    object
dtypes: object(1)
memory usage: 1.7+ KB


# Step 3: Text Preprocessing
- 3.1: Remove Punctuations.
- 3.2: Remove Stopwords.
- 3.3: Stemming / Lemmatization.

In [7]:
import nltk
import re
from nltk.corpus import stopwords
from nltk.stem import WordNetLemmatizer
wnl = WordNetLemmatizer()

corpus=[]
for i in range(len(npr)):
    rp = re.sub("[^a-zA-Z]", " ",npr["Article"][i])
    rp = rp.lower()
    rp = rp.split()
    rp = [wnl.lemmatize(word) for word in rp if not word in stopwords.words("english")] # list list comprehensio (just if in for loop). 
    rp = " ".join(rp)
    corpus.append(rp)

#print(corpus)


# Step 4: Vectorizing

In [8]:
from sklearn.feature_extraction.text import CountVectorizer
cv = CountVectorizer()
dtm = cv.fit_transform(corpus)

# Step 5: Modelling using LDA.

In [9]:
from sklearn.decomposition import LatentDirichletAllocation
LDA = LatentDirichletAllocation(n_components=4)
LDA.fit(dtm)

LatentDirichletAllocation(n_components=4)

In [10]:
len(LDA.components_)

4

In [14]:
cv.get_feature_names_out()

array(['aarp', 'aback', 'abad', ..., 'zoologist', 'zootopia', 'zuma'],
      dtype=object)

In [18]:
#To see top 10 words of each clustor. 
for index,topic in enumerate(LDA.components_):
    print(f'The Top 10 Words for topics #{index}')
    print([cv.get_feature_names_out()[i] for i in topic.argsort()[-10:]])
    print("\n")

The Top 10 Words for topics #0
['time', 'would', 'republican', 'one', 'new', 'year', 'people', 'trump', 'said', 'say']


The Top 10 Words for topics #1
['time', 'state', 'new', 'one', 'like', 'trump', 'people', 'year', 'said', 'say']


The Top 10 Words for topics #2
['president', 'obama', 'said', 'people', 'time', 'one', 'like', 'year', 'trump', 'say']


The Top 10 Words for topics #3
['time', 'counselor', 'like', 'new', 'school', 'student', 'one', 'year', 'people', 'say']




In [19]:
topic_results = LDA.transform(dtm)

In [20]:
topic_results.shape

(200, 4)

In [22]:
#to check which cluster does record one of the data belong to. 
topic_results[0]

array([3.86769087e-04, 3.80175268e-04, 9.98856061e-01, 3.76994713e-04])

In [23]:
topic_results[0].argmax()

2

In [25]:
## Combine the cluster info the initial data frame. 

npr["topic"] = topic_results.argmax(axis=1)
npr.head(10)

,Article,topic
0,"In the Washington of 2016, even when the polic...",2
1,Donald Trump has used Twitter — his prefe...,2
2,Donald Trump is unabashedly praising Russian...,2
3,"Updated at 2:50 p. m. ET, Russian President Vl...",2
4,"From photography, illustration and video, to d...",1
5,I did not want to join yoga class. I hated tho...,0
6,With a who has publicly supported the debunk...,3
7,"I was standing by the airport exit, debating w...",2
8,"If movies were trying to be more realistic, pe...",3
9,"Eighteen years ago, on New Year’s Eve, David F...",2
